In [18]:
import os
import pickle
import numpy as np
import pandas as pd

csv_pose_file = 'pose_urfd.csv'
save_path = 'urfd.pkl'

class_names = ['Not fall', 'Fall']
main_parts = ['LShoulder_x', 'LShoulder_y', 'RShoulder_x', 'RShoulder_y', 'LHip_x', 'LHip_y',
              'RHip_x', 'RHip_y']
main_idx_parts = [1, 2, 7, 8, -1]  # 1.5


# Params.
smooth_labels_step = 8
n_frames = 30
skip_frame = 1

annot = pd.read_csv(csv_pose_file)


In [19]:
num_rows = annot.shape[0]
print("Number of rows in annot:", num_rows)

Number of rows in annot: 1685


In [20]:
idx = annot.iloc[:, 2:-1][main_parts].isna().sum(1) > 0
idx = np.where(idx)[0]
print(idx)
annot = annot.drop(idx)
num_rows = annot.shape[0]
print("Number of rows in annot:", num_rows)


[1135 1552]
Number of rows in annot: 1683


In [22]:
# One-Hot Labels.
label_onehot = pd.get_dummies(annot['label'])
annot = annot.drop('label', axis=1).join(label_onehot)
cols = label_onehot.columns.values

In [71]:


def scale_pose(xy):
    """
    Normalize pose points by scale with max/min value of each pose.
    xy : (frames, parts, xy) or (parts, xy)
    """
    if xy.ndim == 2:
        xy = np.expand_dims(xy, 0)
    xy_min = np.nanmin(xy, axis=1)
    xy_max = np.nanmax(xy, axis=1)
    for i in range(xy.shape[0]):
        xy[i] = ((xy[i] - xy_min[i]) / (xy_max[i] - xy_min[i])) * 2 - 1
    return xy.squeeze()
def seq_label_smoothing(labels, max_step=10):
    steps = 0
    remain_step = 0
    target_label = 0
    active_label = 0
    start_change = 0
    max_val = np.max(labels)
    min_val = np.min(labels)
    for i in range(labels.shape[0]):
        if remain_step > 0:
            if i >= start_change:
                labels[i][active_label] = max_val * remain_step / steps
                labels[i][target_label] = max_val * (steps - remain_step) / steps \
                    if max_val * (steps - remain_step) / steps else min_val
                remain_step -= 1
            continue

        diff_index = np.where(np.argmax(labels[i:i+max_step], axis=1) - np.argmax(labels[i]) != 0)[0]
        if len(diff_index) > 0:
            start_change = i + remain_step // 2
            steps = diff_index[0]
            remain_step = steps
            target_label = np.argmax(labels[i + remain_step])
            active_label = np.argmax(labels[i])
    return labels

In [25]:

feature_set = np.empty((0, n_frames, 14, 3))
labels_set = np.empty((0, len(cols)))
vid_list = annot['video'].unique()

In [26]:
feature_set

array([], shape=(0, 30, 14, 3), dtype=float64)

In [27]:
labels_set

array([], shape=(0, 2), dtype=float64)

In [28]:
annot

,video,frame,Nose_x,Nose_y,Nose_s,LShoulder_x,LShoulder_y,LShoulder_s,RShoulder_x,RShoulder_y,...,RKnee_y,RKnee_s,LAnkle_x,LAnkle_y,LAnkle_s,RAnkle_x,RAnkle_y,RAnkle_s,0,1
0,fall-01,1,0.645445,0.259531,0.935684,0.691758,0.313563,0.886154,0.628078,0.321281,...,0.560563,0.875167,0.697547,0.645469,0.655076,0.645445,0.645469,0.869590,True,False
1,fall-01,2,0.643102,0.259531,0.925543,0.689414,0.313563,0.885889,0.625734,0.321281,...,0.560563,0.849684,0.695203,0.645469,0.673745,0.643102,0.645469,0.861271,True,False
2,fall-01,3,0.647969,0.256146,0.944614,0.688984,0.310833,0.891619,0.624531,0.318646,...,0.560833,0.863793,0.694844,0.646771,0.793520,0.642109,0.646771,0.853266,True,False
3,fall-01,4,0.647422,0.261094,0.952466,0.687617,0.314688,0.863852,0.624453,0.322344,...,0.559688,0.876088,0.699102,0.643906,0.647590,0.647422,0.643906,0.818945,True,False
4,fall-01,5,0.646313,0.258292,0.947201,0.687656,0.313417,0.897403,0.622688,0.321292,...,0.565417,0.884658,0.699469,0.652042,0.716619,0.646313,0.644167,0.852341,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680,fall-30,66,0.310078,0.875911,0.303253,0.277559,1.005990,0.089537,0.284063,0.971302,...,0.858568,0.117682,0.485684,0.971302,0.156369,0.277559,0.988646,0.080249,False,True
1681,fall-30,67,0.319258,0.893490,0.232663,0.280586,0.996615,0.056626,0.280586,0.988021,...,0.841927,0.136853,0.306367,0.902083,0.202637,0.280586,0.988021,0.133989,False,True
1682,fall-30,68,0.433301,0.910807,0.207741,0.447715,0.987682,0.064832,0.281953,0.968464,...,0.833932,0.237444,0.303574,0.891589,0.377427,0.627891,0.853151,0.215639,False,True
1683,fall-30,69,0.423262,0.912214,0.122752,0.269453,1.018854,0.086389,0.287910,0.969635,...,0.846589,0.140247,0.306367,0.895807,0.122043,0.620137,0.838385,0.100483,False,True


In [29]:
vid = vid_list[0]

In [31]:
vid

'fall-01'

In [32]:
print(f'Process on: {vid}')
data = annot[annot['video'] == vid].reset_index(drop=True).drop(columns='video')


Process on: fall-01


In [33]:
data

,frame,Nose_x,Nose_y,Nose_s,LShoulder_x,LShoulder_y,LShoulder_s,RShoulder_x,RShoulder_y,RShoulder_s,...,RKnee_y,RKnee_s,LAnkle_x,LAnkle_y,LAnkle_s,RAnkle_x,RAnkle_y,RAnkle_s,0,1
0,1,0.645445,0.259531,0.935684,0.691758,0.313563,0.886154,0.628078,0.321281,0.942347,...,0.560563,0.875167,0.697547,0.645469,0.655076,0.645445,0.645469,0.869590,True,False
1,2,0.643102,0.259531,0.925543,0.689414,0.313563,0.885889,0.625734,0.321281,0.937608,...,0.560563,0.849684,0.695203,0.645469,0.673745,0.643102,0.645469,0.861271,True,False
2,3,0.647969,0.256146,0.944614,0.688984,0.310833,0.891619,0.624531,0.318646,0.943152,...,0.560833,0.863793,0.694844,0.646771,0.793520,0.642109,0.646771,0.853266,True,False
3,4,0.647422,0.261094,0.952466,0.687617,0.314688,0.863852,0.624453,0.322344,0.927087,...,0.559688,0.876088,0.699102,0.643906,0.647590,0.647422,0.643906,0.818945,True,False
4,5,0.646313,0.258292,0.947201,0.687656,0.313417,0.897403,0.622688,0.321292,0.960016,...,0.565417,0.884658,0.699469,0.652042,0.716619,0.646313,0.644167,0.852341,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,156,0.395576,0.825326,0.270604,0.510010,0.816849,0.134676,0.427363,0.884661,0.366581,...,0.859232,0.162887,0.408291,0.842279,0.244469,0.408291,0.842279,0.271072,False,True
122,157,0.393281,0.815026,0.313128,0.437373,0.873815,0.233435,0.424775,0.882214,0.292854,...,0.873815,0.159534,0.412178,0.840221,0.337166,0.412178,0.840221,0.313878,False,True
123,158,0.393594,0.832812,0.349999,0.437891,0.883438,0.178316,0.431563,0.891875,0.201615,...,0.875000,0.175705,0.412578,0.841250,0.332189,0.412578,0.841250,0.278103,False,True
124,159,0.397480,0.825130,0.346877,0.435801,0.876224,0.177355,0.429414,0.884740,0.358989,...,0.876224,0.182200,0.410254,0.842161,0.275297,0.410254,0.842161,0.307139,False,True


In [36]:
cols

array([0, 1])

In [34]:

# Label Smoothing.
esp = 0.1
data[cols] = data[cols] * (1 - esp) + (1 - data[cols]) * esp / (len(cols) - 1)
data[cols] = seq_label_smoothing(data[cols].values, smooth_labels_step)



In [35]:
data

,frame,Nose_x,Nose_y,Nose_s,LShoulder_x,LShoulder_y,LShoulder_s,RShoulder_x,RShoulder_y,RShoulder_s,...,RKnee_y,RKnee_s,LAnkle_x,LAnkle_y,LAnkle_s,RAnkle_x,RAnkle_y,RAnkle_s,0,1
0,1,0.645445,0.259531,0.935684,0.691758,0.313563,0.886154,0.628078,0.321281,0.942347,...,0.560563,0.875167,0.697547,0.645469,0.655076,0.645445,0.645469,0.869590,0.9,0.1
1,2,0.643102,0.259531,0.925543,0.689414,0.313563,0.885889,0.625734,0.321281,0.937608,...,0.560563,0.849684,0.695203,0.645469,0.673745,0.643102,0.645469,0.861271,0.9,0.1
2,3,0.647969,0.256146,0.944614,0.688984,0.310833,0.891619,0.624531,0.318646,0.943152,...,0.560833,0.863793,0.694844,0.646771,0.793520,0.642109,0.646771,0.853266,0.9,0.1
3,4,0.647422,0.261094,0.952466,0.687617,0.314688,0.863852,0.624453,0.322344,0.927087,...,0.559688,0.876088,0.699102,0.643906,0.647590,0.647422,0.643906,0.818945,0.9,0.1
4,5,0.646313,0.258292,0.947201,0.687656,0.313417,0.897403,0.622688,0.321292,0.960016,...,0.565417,0.884658,0.699469,0.652042,0.716619,0.646313,0.644167,0.852341,0.9,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,156,0.395576,0.825326,0.270604,0.510010,0.816849,0.134676,0.427363,0.884661,0.366581,...,0.859232,0.162887,0.408291,0.842279,0.244469,0.408291,0.842279,0.271072,0.1,0.9
122,157,0.393281,0.815026,0.313128,0.437373,0.873815,0.233435,0.424775,0.882214,0.292854,...,0.873815,0.159534,0.412178,0.840221,0.337166,0.412178,0.840221,0.313878,0.1,0.9
123,158,0.393594,0.832812,0.349999,0.437891,0.883438,0.178316,0.431563,0.891875,0.201615,...,0.875000,0.175705,0.412578,0.841250,0.332189,0.412578,0.841250,0.278103,0.1,0.9
124,159,0.397480,0.825130,0.346877,0.435801,0.876224,0.177355,0.429414,0.884740,0.358989,...,0.876224,0.182200,0.410254,0.842161,0.275297,0.410254,0.842161,0.307139,0.1,0.9


In [92]:
frames = data['frame'].values
frames_set = []
fs = [0]

In [93]:
frames

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82, 115, 117, 118, 119, 120, 121, 122, 123, 124,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
       152, 153, 154, 155, 156, 157, 158, 159, 160])

In [94]:
for i in range(1, len(frames)):
    # print(i, frames[i] < frames[i-1] + 10)
    if frames[i] < frames[i-1] + 10:
        fs.append(i)
    else:
        print(i, "********")
        frames_set.append(fs)
        fs = [i]
frames_set.append(fs)

82 ********


In [95]:
fs = frames_set[0]

In [96]:
fs

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81]

In [97]:
data.shape

(126, 42)

In [98]:
data.iloc[fs, 1:-len(cols)]

,Nose_x,Nose_y,Nose_s,LShoulder_x,LShoulder_y,LShoulder_s,RShoulder_x,RShoulder_y,RShoulder_s,LElbow_x,...,LKnee_s,RKnee_x,RKnee_y,RKnee_s,LAnkle_x,LAnkle_y,LAnkle_s,RAnkle_x,RAnkle_y,RAnkle_s
0,0.645445,0.259531,0.935684,0.691758,0.313563,0.886154,0.628078,0.321281,0.942347,0.714914,...,0.843394,0.633867,0.560563,0.875167,0.697547,0.645469,0.655076,0.645445,0.645469,0.869590
1,0.643102,0.259531,0.925543,0.689414,0.313563,0.885889,0.625734,0.321281,0.937608,0.712570,...,0.861147,0.631523,0.560563,0.849684,0.695203,0.645469,0.673745,0.643102,0.645469,0.861271
2,0.647969,0.256146,0.944614,0.688984,0.310833,0.891619,0.624531,0.318646,0.943152,0.712422,...,0.832305,0.636250,0.560833,0.863793,0.694844,0.646771,0.793520,0.642109,0.646771,0.853266
3,0.647422,0.261094,0.952466,0.687617,0.314688,0.863852,0.624453,0.322344,0.927087,0.710586,...,0.843947,0.635938,0.559688,0.876088,0.699102,0.643906,0.647590,0.647422,0.643906,0.818945
4,0.646313,0.258292,0.947201,0.687656,0.313417,0.897403,0.622688,0.321292,0.960016,0.711281,...,0.843397,0.634500,0.565417,0.884658,0.699469,0.652042,0.716619,0.646313,0.644167,0.852341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,0.554156,0.221625,0.963784,0.619969,0.289875,0.899870,0.539531,0.289875,0.917298,0.641906,...,0.896486,0.554156,0.592125,0.936150,0.612656,0.699375,0.916272,0.576094,0.699375,0.939276
78,0.550250,0.223708,0.968989,0.616063,0.282208,0.897923,0.535625,0.291958,0.941906,0.638000,...,0.903359,0.550250,0.594208,0.948336,0.616063,0.701458,0.905266,0.564875,0.701458,0.930924
79,0.545070,0.221583,0.961532,0.612359,0.281396,0.897845,0.530117,0.291365,0.944593,0.634789,...,0.914618,0.545070,0.590427,0.943853,0.619836,0.700083,0.869505,0.567500,0.710052,0.945923
80,0.539391,0.222458,0.959291,0.607313,0.282833,0.905432,0.524297,0.292896,0.941990,0.637500,...,0.945223,0.539391,0.594771,0.953210,0.614859,0.695396,0.894892,0.562031,0.715521,0.932841


In [99]:
data.iloc[fs, 1:-len(cols)].values.reshape(-1, 13, 3)[0]

array([[0.64544535, 0.25953126, 0.93568355],
       [0.6917578 , 0.31356251, 0.88615358],
       [0.6280781 , 0.32128128, 0.94234741],
       [0.71491408, 0.38303125, 0.89677703],
       [0.61650002, 0.39075002, 0.91543996],
       [0.72070312, 0.45250002, 0.84727484],
       [0.60492194, 0.45250002, 0.90431321],
       [0.68017972, 0.46793753, 0.80636334],
       [0.6338672 , 0.46793753, 0.80632514],
       [0.6917578 , 0.56056255, 0.84339398],
       [0.6338672 , 0.56056255, 0.87516731],
       [0.69754684, 0.64546883, 0.65507579],
       [0.64544535, 0.64546883, 0.86958957]])

In [100]:
xys = data.iloc[fs, 1:-len(cols)].values.reshape(-1, 13, 3)

In [101]:
xys.shape

(82, 13, 3)

In [102]:
xys[0]

array([[0.64544535, 0.25953126, 0.93568355],
       [0.6917578 , 0.31356251, 0.88615358],
       [0.6280781 , 0.32128128, 0.94234741],
       [0.71491408, 0.38303125, 0.89677703],
       [0.61650002, 0.39075002, 0.91543996],
       [0.72070312, 0.45250002, 0.84727484],
       [0.60492194, 0.45250002, 0.90431321],
       [0.68017972, 0.46793753, 0.80636334],
       [0.6338672 , 0.46793753, 0.80632514],
       [0.6917578 , 0.56056255, 0.84339398],
       [0.6338672 , 0.56056255, 0.87516731],
       [0.69754684, 0.64546883, 0.65507579],
       [0.64544535, 0.64546883, 0.86958957]])

In [103]:
xys[:, :, :2] = scale_pose(xys[:, :, :2])

In [104]:
xys[0]

array([[-3.00000103e-01, -1.00000000e+00,  9.35683548e-01],
       [ 4.99999485e-01, -7.20000037e-01,  8.86153579e-01],
       [-6.00001236e-01, -6.79999932e-01,  9.42347407e-01],
       [ 9.00000309e-01, -3.60000173e-01,  8.96777034e-01],
       [-8.00000618e-01, -3.20000068e-01,  9.15439963e-01],
       [ 1.00000000e+00, -1.54441156e-07,  8.47274840e-01],
       [-1.00000000e+00, -1.54441156e-07,  9.04313207e-01],
       [ 3.00000103e-01,  7.99999012e-02,  8.06363344e-01],
       [-5.00000515e-01,  7.99999012e-02,  8.06325138e-01],
       [ 4.99999485e-01,  5.59999926e-01,  8.43393981e-01],
       [-5.00000515e-01,  5.59999926e-01,  8.75167310e-01],
       [ 5.99999176e-01,  1.00000000e+00,  6.55075788e-01],
       [-3.00000103e-01,  1.00000000e+00,  8.69589567e-01]])

In [105]:
xys = np.concatenate((xys, np.expand_dims((xys[:, 1, :] + xys[:, 2, :]) / 2, 1)), axis=1)


In [106]:
xys[0]

array([[-3.00000103e-01, -1.00000000e+00,  9.35683548e-01],
       [ 4.99999485e-01, -7.20000037e-01,  8.86153579e-01],
       [-6.00001236e-01, -6.79999932e-01,  9.42347407e-01],
       [ 9.00000309e-01, -3.60000173e-01,  8.96777034e-01],
       [-8.00000618e-01, -3.20000068e-01,  9.15439963e-01],
       [ 1.00000000e+00, -1.54441156e-07,  8.47274840e-01],
       [-1.00000000e+00, -1.54441156e-07,  9.04313207e-01],
       [ 3.00000103e-01,  7.99999012e-02,  8.06363344e-01],
       [-5.00000515e-01,  7.99999012e-02,  8.06325138e-01],
       [ 4.99999485e-01,  5.59999926e-01,  8.43393981e-01],
       [-5.00000515e-01,  5.59999926e-01,  8.75167310e-01],
       [ 5.99999176e-01,  1.00000000e+00,  6.55075788e-01],
       [-3.00000103e-01,  1.00000000e+00,  8.69589567e-01],
       [-5.00008752e-02, -6.99999985e-01,  9.14250493e-01]])